In [1]:
#https://detectron2.readthedocs.io/
import detectron2
import logging
from detectron2.utils.logger import setup_logger
setup_logger()
logger = logging.getLogger('detectron2')

In [2]:
#Paths
from pathlib import Path
base_folder = Path('..')
data_folder = base_folder/'data'/'df2'
train_imgs_folder = data_folder/'train'
train_annotations = data_folder/'train.json'

save_model_folder = base_folder/'ckpts'
load_model_folder = base_folder/'final_ckpts'

In [3]:
#register datasets. now they can be used as if they were native. remember to run fix_annotations.py to convert TIL2020 to proper COCO format
#to implement custom loaders, such as pickled, https://detectron2.readthedocs.io/modules/data.html?highlight=DatasetCatalog#detectron2.data.DatasetCatalog
from detectron2.data.datasets import register_coco_instances
register_coco_instances("df2_train", {}, train_annotations, train_imgs_folder)
register_coco_instances("til_test", {}, base_folder/'data'/'til2020'/'CV_final_evaluation.json', base_folder/'data'/'til2020'/'CV_final_images')
register_coco_instances("til_val", {}, base_folder/'data'/'til2020'/'val.json', base_folder/'data'/'til2020'/'val')
register_coco_instances("moda_val", {}, base_folder/'data'/'modanet'/'val.json', base_folder/'data'/'modanet'/'train')

In [4]:
#https://detectron2.readthedocs.io/modules/config.html
#btw, i added some custom config options for my checkpointer and pipeline
from detectron2 import model_zoo
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
#cfg.SEED = 42

cfg.DATASETS.TRAIN = ("df2_train",)
cfg.DATASETS.TEST = ("til_val",)
cfg.OUTPUT_DIR = str(save_model_folder)

cfg.DATALOADER.NUM_WORKERS = 16
cfg.DATALOADER.SAMPLER_TRAIN = "RepeatFactorTrainingSampler" #deals with class imbalance
cfg.DATALOADER.REPEAT_THRESHOLD = 0.05

cfg.SOLVER.IMS_PER_BATCH = 4 #batch_size
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.GAMMA = 0.5 #halves the learning rate at each milestone
cfg.SOLVER.STEPS = (100800000,)#(20000,60000,100000,140000) # milestones in iterations
#^I played with these but realised it had little effect: so keep BASE_LR at 0.000125*IMS_PER_BATCH

#Pipeline augmentation settings (i implemented these)
cfg.INPUT.CROP.ENABLED = False
cfg.INPUT.ENABLE_MINOR_AUGMENTS = False
cfg.INPUT.MIN_SIZE_TRAIN = (400,432,464,496,528,560) #mimmicking the original config but adapted for the 400x600 dimension
cfg.INPUT.MIN_SIZE_TEST = 560

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5 #number of categories (remapped the dataset)
cfg.PRINT_EVERY = 200
cfg.SAVE_EVERY = 2000 #when using ValCheckpointer, it saves only if val loss is the minimum so far

cfg.SOLVER.MAX_ITER = 1000000

logger.info(f"Loaded Config:\n{cfg.dump()}")

[06/20 13:26:57 detectron2]: Loaded Config:
CUDNN_BENCHMARK: false
DATALOADER:
  ASPECT_RATIO_GROUPING: true
  FILTER_EMPTY_ANNOTATIONS: true
  NUM_WORKERS: 16
  REPEAT_THRESHOLD: 0.05
  SAMPLER_TRAIN: RepeatFactorTrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: []
  PROPOSAL_FILES_TRAIN: []
  TEST:
  - til_val
  TRAIN:
  - df2_train
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: false
    SIZE:
    - 0.9
    - 0.9
    TYPE: relative_range
  ENABLE_MINOR_AUGMENTS: false
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 560
  MIN_SIZE_TRAIN:
  - 400
  - 432
  - 464
  - 496
  - 528
  - 560
  MIN_SIZE_TRAIN_SAMPLING: choice
MODEL:
  ANCHOR_GENERATOR:
    ANGLES:
    - - -90
      - 0
      - 90
    ASPECT_RATIOS:
    - - 0.5
      - 1.0
      - 2.0
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES:
    - - 32
    - - 64
    - - 128
    - - 256
    - - 5

In [5]:
from detectron2.modeling import build_model
from scripts.trainer import do_train,do_test
model = build_model(cfg)
logger.info(f"Created Model:\n{model}")

 (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
        (2): BottleneckBlock(
          (conv1): Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv2): Conv2d(
            64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv3): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
      )
      (res3): Sequential(
        (0): BottleneckBlock(
          (shortcut): Conv2d(
            256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False
            (norm): FrozenBatchNorm2d(num_features=512, eps=1e-05)
          )
          (conv1): Conv2d(
            256, 128, kernel_size=(1, 1), stride=(2, 2), bias=False
            (n

In [6]:
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml")
#cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn_moda-65999-best_val.pth")
do_train(cfg,model,"ft-til_resnet101_rcnn_df2",resume=True,reset_val=True)
#if this doesnt work correctly, check what model is set in the  file in /ckpts/last_checkpoint
#I uploaded the model file to the google drive

[06/19 15:06:15 detectron2]: Resumed model: ft-til_resnet101_rcnn_df2
[06/19 15:06:15 detectron2]: Pipeline: [ResizeShortestEdge(short_edge_length=(400, 432, 464, 496, 528, 560), max_size=1333, sample_style='choice'), RandomFlip()]
[06/19 15:06:20 d2.data.datasets.coco]: Loading ../data/df2/train.json takes 4.88 seconds.
[06/19 15:06:20 d2.data.datasets.coco]: Loaded 191961 images in COCO format from ../data/df2/train.json
[06/19 15:06:22 d2.data.build]: Removed 522 images with no usable annotations. 191439 images left.
[06/19 15:06:26 d2.data.build]: Distribution of instances among all 5 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    tops    | 107709       |  trousers  | 92003        | outerwear  | 30095        |
|  dresses   | 49559        |   skirts   | 30835        |            |              |
|   total    | 310201       |            |      

In [6]:
#if do_train() hasnt been called (it loads the model) and you want to load the model
if True:
    from detectron2.checkpoint import DetectionCheckpointer
    cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn_df2-91999-best_val.pth")
    DetectionCheckpointer(model, cfg.OUTPUT_DIR).resume_or_load(cfg.MODEL.WEIGHTS, resume=False)

do_test(cfg,model,dataset_name='til_test')

[06/20 13:27:06 d2.data.datasets.coco]: Loaded 1972 images in COCO format from ../data/til2020/CV_final_evaluation.json
[06/20 13:27:06 d2.data.build]: Distribution of instances among all 5 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    tops    | 0            |  trousers  | 0            | outerwear  | 0            |
|  dresses   | 0            |   skirts   | 0            |            |              |
|   total    | 0            |            |              |            |              |
[06/20 13:27:06 d2.data.common]: Serializing 1972 elements to byte tensors and concatenating them all ...
[06/20 13:27:06 d2.data.common]: Serialized dataset takes 0.23 MiB
[06/20 13:27:06 d2.evaluation.evaluator]: Start inference on 1972 images
[06/20 13:27:08 d2.evaluation.evaluator]: Inference done 11/1972. 0.0591 s / img. ETA=0:01:56
[06/20 13:27:13 d2.evaluatio

In [8]:
evaluator = do_test(cfg,model,dataset_name='til_test')

In [ ]:
cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn_moda-65999-best_val.pth")
with open(load_model_folder/f'ft-til_resnet101_rcnn_moda','w') as f:
    f.write(cfg.dump())

In [16]:
import cv2
from IPython.display import display
from PIL import Image
import numpy as np

from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
im = cv2.imread(train_imgs_folder/"69.jpg")
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
im_out = Image.fromarray(v.get_image()[:, :, ::-1]) #channels are reversed
#display(im_out)